### Cálculo visibilidade a partir de um conjunto de pontos de interesse ###

1 - Obter grelha de referência;

2 - Iniciar processo iterativo - para cada célula na grelha de referência;

    2.1 - Obter pontos de observação da célula em causa;
    
    2.2 - Expandir célula em causa, aumentando a sua área em 20 Km, em todas as direcções;
    
    2.3 - Obter DEM para a área obtida em 2.2;
    
    2.4 - Cálculo da visilidade para cada um dos pontos obtidos;
    
    2.5 - Agregação de todos os rasters de visibilidade num só;
    
    2.6 - Eliminação de todos os rasters temporários.

In [ ]:
API_MAIN_URL = 'http://172.16.3.31/api/geodb'
#API_MAIN_URL = 'http://127.0.0.1:8000/api/geodb'

CREDENTIALS = ('kurosaki', 'zangetsu++0012')

obs_dataset = 'obs_fire_clc18'
dem_dataset = 'dem888_10'

CELL_DIM= 20000

workfolder = '/home/jasp/mrgis/obs_fire_clc18'

In [ ]:
import pandas as pd
import os
from osgeo import ogr
from glass.pys.web import data_from_get, data_from_post, get_file
from glass.pys.zzip import unzip
from glass.g.rd.shp import shp_to_obj
from glass.g.gobj import create_polygon
from glass.pys.oss import lst_ff

In [ ]:
# Get ref grid

refgrid = pd.DataFrame(data_from_get(
    "{}/grid/".format(API_MAIN_URL), auth=CREDENTIALS
))

In [ ]:
results = lst_ff(workfolder, file_format='.tif', rfilename=True)

In [ ]:
results = pd.DataFrame(results, columns=['filename'])

In [ ]:
results['name'] = results.filename.str.split('.', expand=True)[0]
results['format'] = results.filename.str.split('.', expand=True)[1]
results['jcellid'] = results.name.str.split('_', expand=True)[1].astype(int)

In [ ]:
refgrid = refgrid.merge(results, how='left', left_on='cellid', right_on='jcellid')

In [ ]:
refgrid = refgrid[refgrid.cellid != refgrid.jcellid]

In [ ]:
refgrid.reset_index(inplace=True)

In [ ]:
refgrid.drop(['index', 'filename', 'name', 'format', 'jcellid'], axis=1, inplace=True)

In [ ]:
print(refgrid.shape)

In [ ]:
refgrid.head(10)

In [ ]:
refgrid = refgrid[refgrid.cellid == 183]

cellid = refgrid.iloc[0].cellid
print(cellid)

geom   = refgrid.iloc[0].geom
print(geom)

In [ ]:
print({
        "dataset" : obs_dataset,
        "geom"    : geom,
        "epsg"    : 3763
    })

In [ ]:
# Get Observation Points Dataset

dataextract = data_from_post(
    "{}/getdata/".format(API_MAIN_URL),
    postdata={
        "dataset" : obs_dataset,
        "geom"    : geom,
        "epsg"    : 3763
    },
    credentials=CREDENTIALS
)

obs_points = get_file("{}/download/{}/".format(
    API_MAIN_URL, dataextract['data']['token']
), os.path.join(workfolder, 'obs_points.zip'), useWget=True)

# Unzip obs points
unzip(obs_points, workfolder)
obs_shp = lst_ff(workfolder, file_format='.shp')[0]

In [ ]:
# Expand Cell Area
geom_env = ogr.CreateGeometryFromWkt(geom).GetEnvelope()

geomexp = create_polygon([
    (geom_env[0] - CELL_DIM, geom_env[3] + CELL_DIM),
    (geom_env[1] + CELL_DIM, geom_env[3] + CELL_DIM),
    (geom_env[1] + CELL_DIM, geom_env[2] - CELL_DIM),
    (geom_env[0] - CELL_DIM, geom_env[2] - CELL_DIM),
    (geom_env[0] - CELL_DIM, geom_env[3] + CELL_DIM)
], api='ogr').ExportToWkt()

print(geomexp)

In [ ]:
# Get DEM

dataextract = data_from_post(
    "{}/getdata/".format(API_MAIN_URL),
    postdata={
        "dataset" : dem_dataset,
        "geom"    : geomexp,
        "epsg"    : 3763
    },
    credentials=CREDENTIALS
)

dem = get_file("{}/download/{}/".format(
    API_MAIN_URL, dataextract['data']['token']
), os.path.join(workfolder, 'dem.tif'), useWget=True)

In [ ]:
from glass.g.wenv.grs import run_grass

In [ ]:



gw = workfolder
    

    
# Start GRASS GIS Session
import grass.script.setup as gsetup
gsetup.init(gbase, gw, loc, 'PERMANENT')
    
from glass.dct.geo.torst   import rst_to_grs, grs_to_rst
from glass.g.rst.surf import grs_viewshed
from glass.g.rst.alg  import rstcalc

grs_dem = rst_to_grs(dem, 'grs_dem', as_cmd=True)



# Produce Viewshed for each obs point
vl = []
for idx, row in obs_df.iterrows():
    vrst = grs_viewshed(
        grs_dem, (row.geometry.x, row.geometry.y),
        "visrst_{}".format(str(idx)),
        max_dist=20000
    )
    
    vl.append(vrst)
    
    if idx ==10:
        break

orst = rstcalc(" + ".join(vl), 'joinvis', api='grass')

frst = grs_to_rst(orst, os.path.join(workfolder, 'teste.tif'))

In [ ]:


from glass.pys.web import data_from_get, data_from_post, get_file

obs_points = get_file("https://fireloc.mat.uc.pt/download/{}/".format(
    API_MAIN_URL, dataextract['data']['token']
), os.path.join(workfolder, 'obs_points.zip'), useWget=True)